In [23]:
import tensorflow as tf
import numpy as np
import zipfile 
import os

In [2]:
!wget --no-check-certificate "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" -O "C:\Clarenti\Data\Project\ML\Program\Dataset\Emotion\happy-or-sad.zip"

--2022-08-11 20:58:34--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 34.101.5.112, 34.101.5.80, 34.101.5.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|34.101.5.112|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: 'C:/Clarenti/Data/Project/ML/Program/Dataset/Emotion/happy-or-sad.zip'

     0K .......... .......... .......... .......... ..........  1% 3.15M 1s
    50K .......... .......... .......... .......... ..........  3% 4.03M 1s
   100K .......... .......... .......... .......... ..........  5% 5.07M 1s
   150K .......... .......... .......... .......... ..........  7% 4.19M 1s
   200K .......... .......... .......... .......... ..........  9% 4.70M 1s
   250K .......... .......... .......... .......... .......... 11% 5.57M 1s
   300K .........

In [10]:
BaseFile = "C:\\Clarenti\\Data\\Project\ML\\Program\\Dataset\\Emotion"

# UNZIP
local_zip = zipfile.ZipFile(os.path.join(BaseFile,"happy-or-sad.zip"), 'r')
local_zip.extractall(BaseFile)
local_zip.close()

In [11]:
# Variables

happy_dir = os.path.join(BaseFile, "happy")
sad_dir = os.path.join(BaseFile, "sad")

In [34]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1/255.,
    horizontal_flip=0.2,
    vertical_flip=0.2,
    shear_range=0.1,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    BaseFile,
    target_size=(150,150),
    class_mode='binary',
    batch_size=25,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    BaseFile,
    target_size=(150,150),
    class_mode='binary',
    batch_size=25,
    subset='validation'
)

Found 64 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


In [58]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = tf.keras.models.Sequential([
    Conv2D(name="Conv1", filters=128, kernel_size=(3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(name="Pool1", pool_size=(2,2)),
    Dropout(0.2),
    Conv2D(name="Conv2", filters=64, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(name="Pool2", pool_size=(2,2)),
    Dropout(0.2),
    Conv2D(name="Conv3", filters=32, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(name="Pool3", pool_size=(2,2)),
    Dropout(0.2),
    Conv2D(name="Conv4", filters=32, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(name="Pool4", pool_size=(2,2)),
    Dropout(0.2),
    Flatten(),
    Dense(name="Hidden1", units=512, activation='relu'),
    Dense(name="Hidden2", units=128, activation='relu'),
    Dense(name="Output", units=1, activation='sigmoid')
])

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=["accuracy"]
)

model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv1 (Conv2D)              (None, 148, 148, 128)     3584      
                                                                 
 Pool1 (MaxPooling2D)        (None, 74, 74, 128)       0         
                                                                 
 dropout_29 (Dropout)        (None, 74, 74, 128)       0         
                                                                 
 Conv2 (Conv2D)              (None, 72, 72, 64)        73792     
                                                                 
 Pool2 (MaxPooling2D)        (None, 36, 36, 64)        0         
                                                                 
 dropout_30 (Dropout)        (None, 36, 36, 64)        0         
                                                                 
 Conv3 (Conv2D)              (None, 34, 34, 32)      

In [59]:
# If the accuracy is not changing, it means the optimizer has found a local minimum for the loss. 
# This may be an undesirable minimum. One common local minimum is to always predict the class with the most number of data points.
# You should use weighting on the classes to avoid this minimum.

# from sklearn.utils import compute_class_weight
# classWeight = compute_class_weight('balanced', np.unique(y), y) 
# classWeight = dict(enumerate(classWeight))

model.fit(train_generator, validation_data=validation_generator, epochs=20)

Epoch 1/20
3/3 [==============================] - 9s 2s/step - loss: 1.2626 - accuracy: 0.3750 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 2/20
3/3 [==============================] - 6s 2s/step - loss: 0.8209 - accuracy: 0.5156 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 3/20
3/3 [==============================] - 6s 2s/step - loss: 0.6962 - accuracy: 0.5000 - val_loss: 0.6930 - val_accuracy: 0.5000
Epoch 4/20
3/3 [==============================] - 7s 3s/step - loss: 0.6831 - accuracy: 0.7344 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 5/20
3/3 [==============================] - 8s 2s/step - loss: 0.6714 - accuracy: 0.5156 - val_loss: 0.6893 - val_accuracy: 0.5000
Epoch 6/20
3/3 [==============================] - 9s 3s/step - loss: 0.7456 - accuracy: 0.5781 - val_loss: 0.6913 - val_accuracy: 0.5000
Epoch 7/20
3/3 [==============================] - 8s 3s/step - loss: 0.6089 - accuracy: 0.8594 - val_loss: 0.7009 - val_accuracy: 0.5000
Epoch 8/20
3/3 [=========================